# Convolutional Neural Network

## Import Module

In [ ]:
# TensorFlow and tf.keras
import tensorflow as tf
from tensorflow.keras import datasets, layers, models

# Helper libraries
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Check if GPU is activate
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

## Preprocessing

### Load the Fashion mnist dataset

![alt text](https://www.tensorflow.org/images/fashion-mnist-sprite.png)
src: https://www.tensorflow.org/tutorials/keras/classification

In [ ]:
# Load fashion mnist dataset
mnist = datasets.fashion_mnist

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

### Define the class

In [ ]:
class_names = ['t_shirt', 'trouser', 'pullover', 'dress', 'coat', 'sandal', 'shirt', 'sneaker', 'bag', 'ankle_boots']

In [ ]:
# View sample data
plt.figure(figsize=(10,10))
for i in range(25):
    plt.subplot(5,5,i+1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(train_images[i], cmap=plt.cm.binary)
    plt.xlabel(class_names[train_labels[i]])
plt.show()

## Data exploration

In [ ]:
# Show data shape
train_images.shape

In [ ]:
train_images

In [ ]:
train_images[0]

In [ ]:
train_images = train_images.reshape(train_images.shape[0], 28, 28, 1)
test_images = test_images.reshape(test_images.shape[0], 28, 28, 1)

In [ ]:
train_images.shape

In [ ]:
train_images

In [ ]:
train_images[0]

In [ ]:
train_images = train_images / 255.0
test_images = test_images / 255.0

## Build the model

![Convolutional Neural Network Architecture](https://upload.wikimedia.org/wikipedia/commons/6/63/Typical_cnn.png)

Source: https://en.wikipedia.org/wiki/Convolutional_neural_network

### Define the model

The following example uses a standard conv-net that has 3 layers with drop-out and batch normalization between each layer.

Src: https://research.google.com/seedbank/seed/fashion_mnist_with_keras_and_tpus

In [ ]:
model = models.Sequential()
model.add(layers.BatchNormalization(input_shape=(28, 28, 1)))
model.add(layers.Conv2D(64, (5, 5), padding='same', activation='elu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(layers.Dropout(0.25))

model.add(layers.BatchNormalization(input_shape=(28, 28, 1)))
model.add(layers.Conv2D(128, (5, 5), padding='same', activation='elu'))
model.add(layers.MaxPooling2D(pool_size=(2, 2)))
model.add(layers.Dropout(0.25))

model.add(layers.Flatten())
model.add(layers.Dense(256))
model.add(layers.Activation('elu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(10))
model.add(layers.Activation('softmax'))

In [ ]:
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.summary()

### Fiting model

In [ ]:
model.fit(train_images, train_labels, epochs=5)

### Testing model

In [ ]:
test_loss, test_acc = model.evaluate(test_images, test_labels, verbose=2)

print('\nTest accuracy:', test_acc)

### Image predict

In [ ]:
predictions = model.predict(test_images)

In [ ]:
predictions[0]

In [ ]:
np.argmax(predictions[0])

In [ ]:
class_names[np.argmax(predictions[0])]

In [ ]:
def plot_image(i, predictions_array, true_label, img):
  predictions_array, true_label, img = predictions_array, true_label[i], img[i]
  plt.grid(False)
  plt.xticks([])
  plt.yticks([])

  plt.imshow(img, cmap=plt.cm.binary)

  predicted_label = np.argmax(predictions_array)
  if predicted_label == true_label:
    color = 'blue'
  else:
    color = 'red'

  plt.xlabel("{} {:2.0f}% ({})".format(class_names[predicted_label],
                                100*np.max(predictions_array),
                                class_names[true_label]),
                                color=color)

def plot_value_array(i, predictions_array, true_label):
  predictions_array, true_label = predictions_array, true_label[i]
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array, color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)

  thisplot[predicted_label].set_color('red')
  thisplot[true_label].set_color('blue')

In [ ]:
i = 0
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images.reshape(test_images.shape[0], 28, 28))
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
i = 12
plt.figure(figsize=(6,3))
plt.subplot(1,2,1)
plot_image(i, predictions[i], test_labels, test_images.reshape(test_images.shape[0], 28, 28))
plt.subplot(1,2,2)
plot_value_array(i, predictions[i],  test_labels)
plt.show()

In [ ]:
# Plot the first X test images, their predicted labels, and the true labels.
# Color correct predictions in blue and incorrect predictions in red.
num_rows = 5
num_cols = 3
num_images = num_rows*num_cols
plt.figure(figsize=(2*2*num_cols, 2*num_rows))
for i in range(num_images):
  plt.subplot(num_rows, 2*num_cols, 2*i+1)
  plot_image(i, predictions[i], test_labels, test_images.reshape(test_images.shape[0], 28, 28))
  plt.subplot(num_rows, 2*num_cols, 2*i+2)
  plot_value_array(i, predictions[i], test_labels)
plt.tight_layout()
plt.show()

## Try your own

### Download images from Internet

In [ ]:
!wget 'https://cdn11.bigcommerce.com/s-rxzabllq/images/stencil/1280x1280/products/910/18045/Kids-Plain-Poly-Fit-Quick_Dry-Tshirt-red__13799.1567089094.jpg?c=2'

### Load images from files

In [ ]:
from PIL import Image, ImageOps
from keras.preprocessing.image import load_img

file_loc = '/content/Kids-Plain-Poly-Fit-Quick_Dry-Tshirt-red__13799.1567089094.jpg?c=2'
img = load_img(file_loc, grayscale=True)
img

## Create Prediction function

In [ ]:
def predict(img):
  new_width  = 28
  new_height = 28
  img = np.array(ImageOps.invert(img.resize((new_width, new_height), Image.ANTIALIAS)))
  img = img / 255
  plt.figure(figsize=(6,3))
  plt.subplot(1,2,1)
  plt.imshow(img, cmap=plt.cm.binary)
  plt.subplot(1,2,2)
  predictions_array = model.predict(img.reshape(1, 28, 28, 1))
  plt.grid(False)
  plt.xticks(range(10))
  plt.yticks([])
  thisplot = plt.bar(range(10), predictions_array[0], color="#777777")
  plt.ylim([0, 1])
  predicted_label = np.argmax(predictions_array)
  plt.show()
  return class_names[np.argmax(model.predict(img.reshape(1, 28, 28, 1)))]

In [ ]:
predict(img)